In [3]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from EVO import DeepNeuralNetwork, EvolutionOptimizer, LogisticRegression

device = torch.device("cuda" if torch.cuda.is_available()
                      else ("mps" if torch.backends.mps.is_available() else "cpu"))

transform = transforms.Compose([
    transforms.ToTensor(),                        # (1, 28, 28) in [0,1]
    transforms.Normalize((0.1307,), (0.3081,)),   # standard MNIST normalisation
    transforms.Lambda(lambda x: x.view(-1))       # flatten to (784,)
])

train_ds = datasets.MNIST(".", train=True,  download=True, transform=transform)
test_ds  = datasets.MNIST(".", train=False, download=True, transform=transform)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=1000)

In [ ]:
# Model + Optimizer
model = DeepNeuralNetwork([784, 64, 10])
model.use_diversity_loss = True
optimizer = EvolutionOptimizer(model)
optimizer.set_population_size(10)
optimizer.use_backprop = False
optimizer.set_diversity_coeff(0.1)
optimizer.set_survivors_ratio(0.1)
optimizer.set_fitness_ratio(0.5)
optimizer.set_sneaker_prob(0.1)
optimizer.set_mutation_intensity(0.2)
optimizer.mutation_rate = 0.3

# Evolution loop
for generation in range(10):  # evolve for 10 generations
    for X, y in train_loader:
        optimizer.step(X, y)

    # Test accuracy
    correct = total = 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(model.device), y.to(model.device)   # Move to model's device
            preds = model.predict(X)
            correct += (preds == y).sum().item()
            total   += y.size(0)

    print(f"Generation {generation+1}, Test Accuracy: {100*correct/total:.2f}%")

Generation 1, Test Accuracy: 31.38%
Generation 2, Test Accuracy: 53.15%
Generation 3, Test Accuracy: 52.49%
Generation 4, Test Accuracy: 60.48%
